In [ ]:
import pickle
import numpy as np
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from skbio.stats.composition import clr

# ─── Load Data ───────────────────────────────────────────────────────────────
dataset_path_dict = {
    "embeddings": "/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl",
    "neural": "/home/maria/LuckyMouse2/pixel_transformer_neuro/data/processed/hybrid_neural_responses_reduced.npy"
}

with open(dataset_path_dict['embeddings'], "rb") as f:
    embeddings_raw = pickle.load(f)
embeddings = embeddings_raw['natural_scenes']  # shape: (118, 1000)
print("Full embedding shape:", embeddings.shape)

neural_data = np.load(dataset_path_dict["neural"])  # shape: (neurons, 118)

# ─── Split Train/Test ────────────────────────────────────────────────────────
n_train = 100
X_train_raw = embeddings[:n_train]
X_test_raw = embeddings[n_train:]

# ─── Fit PCA only on training set ────────────────────────────────────────────
pca = PCA(n_components=50, whiten=True)
X_train = pca.fit_transform(X_train_raw)
X_test = pca.transform(X_test_raw)
print("Train PCA shape:", X_train.shape)
print("Test PCA shape:", X_test.shape)

# ─── Add Intercept ───────────────────────────────────────────────────────────
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# ─── Fit Model ───────────────────────────────────────────────────────────────
n_neurons = neural_data.shape[0]
n_trials = 50
r2_test_scores = []

for i in range(n_neurons):
    counts = neural_data[i]

    # Split target variable
    y_train = np.clip(np.round(counts[:n_train]), 0, n_trials)
    y_test = np.clip(np.round(counts[n_train:]), 0, n_trials)
    y_train_binomial = np.column_stack([y_train, n_trials - y_train])

    try:
        model = sm.GLM(y_train_binomial, X_train, family=sm.families.Binomial())
        result = model.fit_regularized(alpha=10.0, L1_wt=0)

        probs_pred = result.predict(X_test)
        counts_pred = n_trials * probs_pred
        r2_count = r2_score(y_test, counts_pred)
        r2_test_scores.append(r2_count)

        probs_true = y_test / n_trials
        r2_prob = r2_score(probs_true, probs_pred)

        print(f"Neuron {i}: R² (counts) = {r2_count:.4f} | R² (probs) = {r2_prob:.4f}")
        print("True counts:", y_test)
        print("Predicted counts:", counts_pred.round(2))

    except Exception as e:
        print(f"Neuron {i}: Error - {e}")
        r2_test_scores.append(None)
